In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import skimage.measure

import random
import copy
import math
import cv2
import os
import sys

import argparse
import datetime
import json
import time
from pathlib import Path

import torch

import models_mae
import util.misc as misc
from engine_pretrain import train_one_epoch
from main_ViT import main, get_args_parser, run_one_image, DATASET_PATH, SemanticMapDataset
import timm.optim.optim_factory as optim_factory
from util.misc import NativeScalerWithGradNormCount as NativeScaler

from scipy.stats import wasserstein_distance

# TRAINED MODELS
## Variables definition

In [ ]:
div = 64
step = int(32/4)
red = 1

mask_ratio = 0.75
test_name = 'test8'
sigma = 2 #3
n_crops = 1000

map_list_t = ['master_big','master_big_closed','master_big_semiclosed']
map_root_name = 'master_big'
map_list = ['stanford_coupa0', 'stanford_coupa1', 'stanford_coupa2', 'stanford_coupa3', 'stanford_deathCircle0', 'stanford_gates2', 'stanford_hyang2', 'stanford_hyang3', 'stanford_hyang4', 'stanford_hyang10', 'stanford_little3', 'stanford_nexus0', 'stanford_nexus1']

sem_dict = ['cash', 'entrance', 'light', 'sit', 'stairs', 'trash', 'tree','restricted','grass','intersection','shadow']
chans = len(sem_dict)+1

lut_in = [0, 10, 30, 50, 70, 110, 150, 255]
lut_out = [0, 120, 200, 225, 240, 248, 250, 255]
lut_in = [0, 10, 70, 150, 255]
lut_out = [0, 130, 210, 245, 255]
# plt.plot(lut_in, lut_out)
lut_8u = np.interp(np.arange(0, 256), lut_in, lut_out).astype(np.uint8)

# Loading of the models

In [ ]:
# args = get_args_parser()
# args = args.parse_args()

# args.resume = '/home/placido.falqueto/IRI_Barcelona/mae/output_dir/test3-checkpoint-100.pth'

# device = torch.device(args.device)

# # define the model
# model = models_mae.__dict__[args.model](norm_pix_loss=args.norm_pix_loss)

# model.to(device)

# model_without_ddp = model

# eff_batch_size = args.batch_size * args.accum_iter * misc.get_world_size()

# if args.lr is None:  # only base_lr is specified
#     args.lr = args.blr * eff_batch_size / 256

# # following timm: set wd as 0 for bias and norm layers
# param_groups = optim_factory.add_weight_decay(model_without_ddp, args.weight_decay)
# optimizer = torch.optim.AdamW(param_groups, lr=args.lr, betas=(0.9, 0.95))
# loss_scaler = NativeScaler()

# misc.load_model(args=args, model_without_ddp=model_without_ddp, optimizer=optimizer, loss_scaler=loss_scaler)

## Testing closed corridors (completely or partially closed)

In [ ]:
# red1 = 8

# for map_name in map_list_t:
#   print(map_name)

#   lines = 0
#   try:
#     with open('/home/placido.falqueto/IRI_Barcelona/maps/semantics/'+map_root_name+'/'+map_name+'-reduced.csv') as f:
#       print('Reduced map found')
#       lines = f.readlines()
      
#   except FileNotFoundError:
#     with open('/home/placido.falqueto/IRI_Barcelona/maps/semantics/'+map_root_name+'/'+map_name+'.csv') as f:
#       print('Using original map')
#       lines = f.readlines()

#   h = len(lines)
#   w = len(lines[0].split(','))

#   # Converts data to a list of integers
#   map = []
#   for line in lines:
#     map.extend([int(c) for c in line.split(',')])

#   for lab_class in sem_dict:
#     lines = 0
#     try:
#       with open('/home/placido.falqueto/IRI_Barcelona/maps/semantics/'+map_name+'/'+map_name+'_sem_'+lab_class+'-reduced.csv') as f:
#         lines = f.readlines()

#       hh = len(lines)
#       ww = len(lines[0].split(','))

#       if hh != h or ww != w:
#         print(f'h: {h}\tw: {w}')
#         print(f'h: {hh}\tw: {ww}')
#         raise SystemExit("ERROR: Different sizes!!")

#       # Converts data to a list of integers
#       for line in lines:
#         map.extend([int(c) for c in line.split(',')])
    
#     except FileNotFoundError:
#       try:
#         with open('/home/placido.falqueto/IRI_Barcelona/maps/semantics/'+map_root_name+'/'+map_name+'_sem_'+lab_class+'.csv') as f:
#           lines = f.readlines()

#         hh = len(lines)
#         ww = len(lines[0].split(','))

#         if hh != h or ww != w:
#           print(f'h: {h}\tw: {w}')
#           print(f'h: {hh}\tw: {ww}')
#           raise SystemExit("ERROR: Different sizes!!")

#         # Converts data to a list of integers
#         for line in lines:
#           map.extend([int(c) for c in line.split(',')])

#       except FileNotFoundError:
#         for i in range(h):
#           for j in range(w):
#             map.extend([255])

#   map = np.reshape(map,[chans,h,w])
#   map = np.moveaxis(map, 0, -1)
#   map = map/255

#   map_aux = map
#   map = np.zeros((int(math.ceil(h/red1)),int(math.ceil(w/red1)),chans))

#   for idx in range(chans):
#     map[:,:,idx] = skimage.measure.block_reduce(map_aux[:,:,idx], (red1,red1), np.max)

#   # print(map.shape)
#   h, w, _ = map.shape

#   lines = 0
#   with open('/home/placido.falqueto/IRI_Barcelona/maps/semantics/'+map_root_name+'/humandensity-'+map_root_name+'-new.csv') as f:
#     lines = f.readlines()

#   hd = len(lines)
#   wd = len(lines[0].split(','))

#   # Converts data to a list of integers
#   data = []
#   for line in lines:
#     data.extend([int(c) for c in line.split(',')])

#   data = np.reshape(data,[hd,wd])
  
#   sigma = 10.0
#   # data = skimage.filters.gaussian(data, sigma=(sigma, sigma), channel_axis=-1)
#   data = skimage.measure.block_reduce(data, (red1,red1), np.max)
#   # data = np.subtract(data, np.full((h, w), np.min(data)))/(np.max(data)-np.min(data))
#   data = data*(map[:,:,0]>0)
  
#   lim_val = 0.15
#   sigma = 0.5
#   data_show = np.clip(data,0,np.max(data)*lim_val)/(np.max(data)*lim_val)
#   data_show = cv2.LUT((data_show*255).astype(np.uint8), lut_8u).astype(np.float32)/255
#   data_show = skimage.filters.gaussian(data_show, sigma=(sigma, sigma), channel_axis=-1)
#   data = data_show/np.sum(data_show)

#   # -----------------------------------------------------------------------------------
#   # Random sample 32x32 windows in map
#   n_crops = 100
#   inserted = 0
#   crops = np.zeros((n_crops,2),dtype=int)
#   selections = np.zeros((h,w))
#   for i in range(n_crops):
#     background = np.zeros((h,w))
#     flag = True
#     while flag:
#       aux_x = int(random.random()*(w-div+1))
#       aux_y = int(random.random()*(h-div+1))
#       flag = False
#       # print(f'INSERTED: {inserted}  ->  x: {aux_x} and y: {aux_y}')
#       # for z in range(inserted):
#       #   if abs(aux_x-crops[z,0])<int(div/10) and abs(aux_y-crops[z,1])<int(div/10):
#       #     # print('too close')
#       #     flag = True
#     crops[i,0] = aux_x
#     crops[i,1] = aux_y
#     selections = selections+cv2.rectangle(background,(crops[i,0],crops[i,1]),(crops[i,0]+div-1,crops[i,1]+div-1),(1,1,1),-1)
#     inserted = inserted + 1
#   selections = 1/selections

#   # -----------------------------------------------------------------------------------
#   # creating subplot and figure
#   ax = plt.subplot(111)
#   data_pred = np.zeros((int(math.ceil(h)),int(math.ceil(w))))
#   kls = []
#   for i in range(n_crops):
#     submap = map[crops[i,1]:crops[i,1]+div, crops[i,0]:crops[i,0]+div,:]
#     data_aux = data[crops[i,1]:crops[i,1]+div, crops[i,0]:crops[i,0]+div]
#     submap = torch.tensor(np.expand_dims(submap,axis=0), dtype=torch.float32).to(device)
#     _, subdata, _ = model(submap)
#     subdata = torch.einsum('nchw->nhwc', subdata).detach().cpu()
#     subdata = subdata.squeeze(dim=0).squeeze(dim=2).numpy()
#     data_pred[crops[i,1]:crops[i,1]+div, crops[i,0]:crops[i,0]+div] += subdata*selections[crops[i,1]:crops[i,1]+div, crops[i,0]:crops[i,0]+div]
    
#   data = torch.Tensor(data)
#   data_pred = torch.Tensor(data_pred)
#   data = data/data.max()
#   data_pred = data_pred/data_pred.max()
  
#   data_pred = data_pred/np.max(data_pred.detach().cpu().numpy())
#   plt.imshow(np.multiply(np.stack((map[:,:,0],map[:,:,0],map[:,:,0]),axis=2), np.stack((np.full(data_pred.shape,1),1-data_pred,1-data_pred),axis=2)), vmin=0, vmax=1)
#   # plt.imshow(np.multiply(np.stack((map[:,:,0],map[:,:,0],map[:,:,0]),axis=2), np.stack((np.full(data_pred.shape,1),1-data_pred,1-data_pred),axis=2)), vmin=0, vmax=1)
#   ax.get_xaxis().set_visible(False)
#   ax.get_yaxis().set_visible(False)
#   plt.show()

In [ ]:
def show_image(features, data, title=''):
    # features is [H, W, C]
    # plt.imshow(torch.clip((features * imagenet_std + imagenet_mean) * 255, 0, 255).int())
    data = data / data.max()
    test11 = np.stack((features[:,:,0],features[:,:,0],features[:,:,0]),axis=2)
    test11 = np.multiply(test11,np.stack((np.full(data.shape,1),1-data,1-data),axis=2))
    for i in range(10):
        alp = 0.5
        test11 = np.multiply(test11,np.stack((features[:,:,i+1],features[:,:,i+1],features[:,:,i+1]),axis=2)*alp+(1-alp))
    plt.imshow(test11)
    plt.title(title, fontsize=23)
    plt.axis('off')
    return

def run_one_image(x, target, model):
    # run MAE
    x = x.to('cuda')
    target = target.to('cuda')
    _, y, mask = model(x, mask_ratio=mask_ratio)
    x = x.detach().cpu()
    target = target.detach().cpu()
    
    y = y.squeeze(3).detach().cpu()

    is_mae = 0
    if mask is not None:
        # visualize the mask
        mask = mask.detach()
        mask = mask.unsqueeze(-1).repeat(1, 1, model.patch_embed.patch_size[0]**2 *1)  # (N, H*W, p*p*3)
        mask = model.unpatchify(mask)  # 1 is removing, 0 is keeping
        mask = torch.einsum('nchw->nhwc', mask).detach().cpu()
        mask = mask.squeeze(-1)
        mask = mask.swapaxes(1, 2)

        plt.figure()
        plt.imshow(mask[0])
        plt.show()

        # masked image
        im_masked = target * (1 - mask) + mask*target.max()

        is_mae = 1

    # # MAE reconstruction pasted with visible patches
    # im_paste = x * (1 - mask) + y * mask

    # make the plt figure larger
    plt.rcParams['figure.figsize'] = [24, 12] # W: 24, H: 12

    plt.subplot(1, 2+is_mae, 1)
    show_image(x[0], target[0], "original")

    if is_mae:
        plt.subplot(1, 2+is_mae, 2)
        show_image(x[0], im_masked[0], "masked")

    plt.subplot(1, 2+is_mae, 2+is_mae)
    show_image(x[0], y[0], "prediction")

    # plt.subplot(1, 4, 3)
    # plt.imshow(target[0])
    # plt.title('target', fontsize=23)
    # plt.axis('off')

    # plt.subplot(1, 4, 4)
    # plt.imshow(y[0])
    # plt.title('y', fontsize=23)
    # plt.axis('off')

    plt.show()

## Model 1 testing

In [ ]:
kl1 = []
kli1 = []
was1 = []
mse1 = []

for map_count, map_name in enumerate(map_list):
  print(map_name)

  args = get_args_parser()
  args = args.parse_args()

  args.resume = '/data/placido/'+test_name+'-checkpoint-100-'+map_name+'.pth'

  device = torch.device(args.device)

  # define the model
  model = models_mae.__dict__[args.model](norm_pix_loss=args.norm_pix_loss)

  model.to(device)

  model_without_ddp = model

  eff_batch_size = args.batch_size * args.accum_iter * misc.get_world_size()

  if args.lr is None:  # only base_lr is specified
      args.lr = args.blr * eff_batch_size / 256

  # following timm: set wd as 0 for bias and norm layers
  param_groups = optim_factory.add_weight_decay(model_without_ddp, args.weight_decay)
  optimizer = torch.optim.AdamW(param_groups, lr=args.lr, betas=(0.9, 0.95))
  loss_scaler = NativeScaler()

  misc.load_model(args=args, model_without_ddp=model_without_ddp, optimizer=optimizer, loss_scaler=loss_scaler)

  lines = 0
  try:
    with open('/home/placido.falqueto/IRI_Barcelona/maps/semantics/'+map_name+'/'+map_name+'-reduced.csv') as f:
      print('Reduced map found')
      lines = f.readlines()
      
  except FileNotFoundError:
    with open('/home/placido.falqueto/IRI_Barcelona/maps/semantics/'+map_name+'/'+map_name+'.csv') as f:
      print('Using original map')
      lines = f.readlines()

  h = len(lines)
  w = len(lines[0].split(','))

  # Converts data to a list of integers
  map = []
  for line in lines:
    map.extend([int(c) for c in line.split(',')])

  for lab_class in sem_dict:
    lines = 0
    try:
      with open('/home/placido.falqueto/IRI_Barcelona/maps/semantics/'+map_name+'/'+map_name+'_sem_'+lab_class+'-reduced.csv') as f:
        lines = f.readlines()

      hh = len(lines)
      ww = len(lines[0].split(','))

      if hh != h or ww != w:
        print(f'h: {h}\tw: {w}')
        print(f'h: {hh}\tw: {ww}')
        raise SystemExit("ERROR: Different sizes!!")

      # Converts data to a list of integers
      for line in lines:
        map.extend([int(c) for c in line.split(',')])
    
    except FileNotFoundError:
      try:
        with open('/home/placido.falqueto/IRI_Barcelona/maps/semantics/'+map_name+'/'+map_name+'_sem_'+lab_class+'.csv') as f:
          lines = f.readlines()

        hh = len(lines)
        ww = len(lines[0].split(','))

        if hh != h or ww != w:
          print(f'h: {h}\tw: {w}')
          print(f'h: {hh}\tw: {ww}')
          raise SystemExit("ERROR: Different sizes!!")

        # Converts data to a list of integers
        for line in lines:
          map.extend([int(c) for c in line.split(',')])

      except FileNotFoundError:
        for i in range(h):
          for j in range(w):
            map.extend([255])

  map = np.reshape(map,[chans,h,w])
  map = np.moveaxis(map, 0, -1)
  map = map/255

  map_aux = map
  map = np.zeros((int(math.ceil(h/red)),int(math.ceil(w/red)),chans))

  for idx in range(chans):
    map[:,:,idx] = skimage.measure.block_reduce(map_aux[:,:,idx], (red,red), np.max)

  # print(map.shape)
  h, w, _ = map.shape

# -----------------------------------------------------------------------------------
  lines = 0
  with open('/home/placido.falqueto/IRI_Barcelona/maps/semantics/'+map_name+'/humandensity-'+map_name+'-new.csv') as f:
    lines = f.readlines()

  hd = len(lines)
  wd = len(lines[0].split(','))

  # Converts data to a list of integers
  data = []
  for line in lines:
    data.extend([int(c) for c in line.split(',')])

  data = np.reshape(data,[hd,wd])
  # data = skimage.filters.gaussian(data, sigma=(sigma, sigma), channel_axis=-1)
  data = skimage.measure.block_reduce(data, (red,red), np.max)
  # data = np.subtract(data, np.full((h, w), np.min(data)))/(np.max(data)-np.min(data))
  data = data*(map[:,:,0]>0)

  data_show = data
  # data_show = cv2.LUT((data_show*255).astype(np.uint8), lut_8u).astype(np.float32)/255
  data_show = skimage.filters.gaussian(data_show, sigma=(sigma, sigma), channel_axis=-1)
  data_show = skimage.filters.gaussian(data_show, sigma=(sigma, sigma), channel_axis=-1)
  data = data_show/np.sum(data_show)
  data_show = data_show/np.max(data_show)

  # print(data.shape)
  hd, wd = data.shape

  print(sem_dict)
  plt.figure(figsize=(20,5))
  for i in range(len(sem_dict)):
    ax = plt.subplot(1, len(sem_dict), i+1)
    alp = 0.5
    ax.imshow(np.multiply(np.stack((map[:,:,0],map[:,:,0],map[:,:,0]),axis=2),np.multiply(np.stack((map[:,:,i+1],map[:,:,i+1],map[:,:,i+1]),axis=2)*alp+(1-alp), np.stack((np.full(data_show.shape,1),1-data_show,1-data_show),axis=2))), vmin=0, vmax=1)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
  plt.show()

  plt.figure()
  plt.imshow(np.multiply(np.stack((map[:,:,0],map[:,:,0],map[:,:,0]),axis=2), np.stack((np.full(data_show.shape,1),1-data_show,1-data_show),axis=2)), vmin=0, vmax=1)
  plt.axis('off')
  plt.show()

  # -----------------------------------------------------------------------------------
  # Random sample 32x32 windows in map
  inserted = 0
  crops = np.zeros((n_crops,2),dtype=int)
  selections = np.zeros((h,w))
  for i in range(n_crops):
    background = np.zeros((h,w))
    flag = True
    while flag:
      aux_x = int(random.random()*(w-div+1))
      aux_y = int(random.random()*(h-div+1))
      flag = False
      # print(f'INSERTED: {inserted}  ->  x: {aux_x} and y: {aux_y}')
      # for z in range(inserted):
      #   if abs(aux_x-crops[z,0])<int(div/10) and abs(aux_y-crops[z,1])<int(div/10):
      #     # print('too close')
      #     flag = True
    crops[i,0] = aux_x
    crops[i,1] = aux_y
    selections = selections+cv2.rectangle(background,(crops[i,0],crops[i,1]),(crops[i,0]+div-1,crops[i,1]+div-1),(1,1,1),-1)
    inserted = inserted + 1
  selections = 1/selections

  # -----------------------------------------------------------------------------------
  # creating subplot and figure
  # ax = plt.subplot(111)
  data_pred = np.zeros((int(math.ceil(h)),int(math.ceil(w))))
  kls = []
  for i in range(n_crops):
    submap = map[crops[i,1]:crops[i,1]+div, crops[i,0]:crops[i,0]+div,:]
    data_aux = data[crops[i,1]:crops[i,1]+div, crops[i,0]:crops[i,0]+div]
    submap = torch.tensor(np.expand_dims(submap,axis=0), dtype=torch.float32).to(device)
    data_aux = torch.tensor(np.expand_dims(data_aux,axis=0), dtype=torch.float32).to(device)
    run_one_image(submap, data_aux, model)
    import sys
    sys.exit(0)
    _, subdata, _ = model(submap, mask_ratio=mask_ratio)
    subdata = torch.einsum('nchw->nhwc', subdata).detach().cpu()
    subdata = subdata.squeeze(dim=0).squeeze(dim=2).numpy()
    data_pred[crops[i,1]:crops[i,1]+div, crops[i,0]:crops[i,0]+div] += subdata*selections[crops[i,1]:crops[i,1]+div, crops[i,0]:crops[i,0]+div]
    # # Calculate the KL-divergence
    # data_aux = torch.Tensor(data_aux/np.sum(data_aux))
    # subdata_pred = torch.Tensor(subdata/np.sum(subdata))
    # kl = torch.nn.functional.kl_div(subdata_pred, data_aux, reduction='mean', log_target=True).item()
    # kls.append(kl)

  data = torch.Tensor(data)
  data_pred = torch.Tensor(data_pred)
  data = data/data.sum()
  data_pred = data_pred/data_pred.sum()
  data_pred[data_pred == 0] = 1e-12
  data_pred[data_pred == 1] = 1-1e-12
  data[data == 0] = 1e-12
  data[data == 1] = 1-1e-12

  kl = torch.nn.functional.kl_div(data_pred.unsqueeze(0).log(), data.unsqueeze(0), reduction='batchmean').item()
  kli = torch.nn.functional.kl_div(data.unsqueeze(0).log(), data_pred.unsqueeze(0), reduction='batchmean').item()
  h, w = data.shape
  data_aux = (data / data.max() * 255).detach().cpu().numpy().astype(int)
  pred_aux = (data_pred / data_pred.max() * 255).detach().cpu().numpy().astype(int)
  hist_a = [0.0] * 256
  for i in range(h):
    for j in range(w):
      hist_a[data_aux[i, j]] += 1
  hist_b = [0.0] * 256
  for i in range(h):
    for j in range(w):
      hist_b[pred_aux[i, j]] += 1
  was = wasserstein_distance(hist_a, hist_b)
  mse = ((data - data_pred)**2).mean()

  data = data/data.max()
  data_pred = data_pred/data_pred.max()
  # plt.figure(figsize=(4,4))
  plt.figure()
  plt.imshow(np.multiply(np.stack((1-data,1-data,np.full(data.shape,1)),axis=2), np.stack((np.full(data_pred.shape,1),1-data_pred,1-data_pred),axis=2)))
  plt.axis('off')
  plt.show()
  
  data_pred = data_pred/np.max(data_pred.detach().cpu().numpy())
  plt.figure()
  plt.imshow(np.multiply(np.stack((map[:,:,0],map[:,:,0],map[:,:,0]),axis=2), np.stack((np.full(data_pred.shape,1),1-data_pred,1-data_pred),axis=2)), vmin=0, vmax=1)
  # plt.imshow(np.multiply(np.stack((map[:,:,0],map[:,:,0],map[:,:,0]),axis=2), np.stack((np.full(data_pred.shape,1),1-data_pred,1-data_pred),axis=2)), vmin=0, vmax=1)
  plt.axis('off')
  plt.show()

  print(f'KL-divergence: {kl}')
  print(f'Reverse KL-divergence: {kli}')
  print(f'EMD: {was:.2f}')
  print(f'MSE: {mse:.2e}')
  kl1 = np.append(kl1,kl)
  kli1 = np.append(kli1,kli)
  was1 = np.append(was1,was)
  mse1 = np.append(mse1,mse)
  # import sys
  # sys.exit(0)

print('-----------------------------')
print(f'Mean KL-divergence: {np.mean(kl1)}')
print(f'Std KL-divergence: {np.std(kl1)}')
print(f'Mean reverse KL-divergence: {np.mean(kli1)}')
print(f'Std reverse KL-divergence: {np.std(kli1)}')
print(f'Mean EMD: {np.mean(was1):.2e}')
print(f'Std EMD: {np.std(was1):.2e}')

# plot all metrics normalized
plt.figure(figsize=(10,5))
plt.plot(kl1 / np.max(kl1), label='KL-divergence', marker='o') #linestyle='None'
plt.plot(kli1 / np.max(kli1), label='Reverse KL-divergence', marker='x')
plt.plot(was1 / np.max(was1), label='EMD', marker='^')
plt.plot(mse1 / np.max(mse1), label='MSE', marker='*')
plt.legend()
plt.show()